# CSCI 3155 Spring 2023
## Recitation Week 14: Traits and Generics

## Traits

Traits are an important mechanism for code reuse in scala. They allow us to define functionality that can be 
exported across multiple objects in the overall hierarchy. A trait is almost like an abstract class or an interface. It can define its own members and methods.

For this exercise, we have defined two traits: `NumberOfLegs` that helps us define how many legs a given animal has and `WarmBlooded` that applies to warm blooded animals.  We also have an abstract class `Animal` as a superclass for all animals.

In [1]:
abstract class Animal

trait NumberOfLegs {
    val nLegs: Int
    def getNumberOfLegs: Int = nLegs
}

trait WarmBlooded extends Animal {
    val bodyTempMaintained: Double
    def getBodyTemp: Double = bodyTempMaintained 
}

defined class Animal
defined trait NumberOfLegs
defined trait WarmBlooded

### Example
We will define a class `Human` which will extend the appropriate classes and traits defined above.
Our class will also take in a parameter called `name` of type `String` and implement a `getName` method without any parameters.

Which traits should we use for our human class? (Discuss this first then move onto the coding of the class)

In [ ]:
//BEGIN SOLUTION
class Human(val name: String) extends WarmBlooded with NumberOfLegs {
    override val nLegs:Int = 2
    val bodyTempMaintained = 98.0
    def getName: String = name
    // below is also valid, but may change testing needs
    // def getName(): String = name
}
//END SOLUTION

In [ ]:
//BEGIN TEST
val t1 = new Human("Jane Smith")
t1.isInstanceOf[Animal]
assert(t1.getNumberOfLegs == 2, "Your human does not have two legs")
assert(t1.bodyTempMaintained == 98.0, "Your human does not maintain a body temp of 98")
assert(t1.getBodyTemp == 98.0, "Your human's getBodyTemp Function is not working")
assert(t1.getName == "Jane Smith", "Your human's name is not setting correctly")
//END TEST

In [2]:
// ALTERNATE SOLUTION
class Human(val name: String, 
            val nLegs: Int = 2, 
            val bodyTempMaintained: Double = 98.0) extends WarmBlooded 
        with NumberOfLegs { 
    def getName(): String = this.name
}
val t1 = new Human("Jane Smith")
t1.isInstanceOf[Animal]
assert(t1.getNumberOfLegs == 2, "Your human does not have two legs")
assert(t1.bodyTempMaintained == 98.0, "Your human does not maintain a body temp of 98")
assert(t1.getBodyTemp == 98.0, "Your human's getBodyTemp Function is not working")
assert(t1.getName == "Jane Smith", "Your human's name is not setting correctly")

defined class Human
t1: Human = ammonite.$sess.cmd1$Helper$Human@52d2fec6
res1_2: Boolean = true

### Exercise
Now we will define a class named `Table`.

Which of the traits and classes should we extend? (discuss this first then advance to the coding) (if someone complains about the number of legs on a table, adjust the exercise so that the table class takes in the number of legs, and update the test)

In [ ]:
//BEGIN SOLUTION
class Table extends NumberOfLegs {
    val nLegs = 4
}
//END SOLUTION

In [ ]:
//BEGIN TEST
val tbl = new Table()
assert(tbl.getNumberOfLegs == 4, "A Table must have four legs")
//END TEST

In [3]:
// ALTERNATE SOLUTION
class Table(val nLegs: Int = 4) extends NumberOfLegs
val tbl = new Table()
assert(tbl.getNumberOfLegs == 4, "A Table must have four legs")

defined class Table
tbl: Table = ammonite.$sess.cmd2$Helper$Table@6ada9223

## Type Casting
We may want to do certain things based on a more precise type than a trait gives us, like only the humans in a list of instances of NumberOfLegs.
While not as easy as pattern matching with case classes, this is possible by using the `isInstanceOf[T]` and the `asInstanceOf[T]` functions.

The expression `obj.isInstanceOf[T]` returns true if `obj` is an instance of `T` and false otherwise.
Similarly, `obj.asInstanceOf[T]` returns the same instance `obj`, but with the given type `T` (if it is an instance of `T`).

### Demo
Consider the following code and it's errors

In [ ]:
// // fails 
// val a0: NumberOfLegs = new Human("Liskov")
// a0.getName

In [ ]:
// works
val a0: NumberOfLegs = new Human("Liskov")
assert(a0.isInstanceOf[Human])
val h0: Human = a0.asInstanceOf[Human]
val name: String = h0.getName

In [ ]:
// // fails
// val a1: NumberOfLegs = new Table()
// assert(a1.isInstanceOf[Human])

### Exercise
To try out these functions, lets write a function to get the names of all the humans in a list of instances of the NumberOfLegs trait.

In [ ]:
def getNames(leggedThings : List[NumberOfLegs]): List[String] = {
    leggedThings match {
        case h :: t =>
            // begin solution
            if (h.isInstanceOf[Human]) 
                (h.asInstanceOf[Human].getName):: getNames(t)
            else 
                getNames(t)
            // end solution
        case Nil => Nil
    }
}

In [ ]:
// TEST CASE
val leggedThings : List[NumberOfLegs] = List(new Human("Charles"), new Human("Sukanya"), new Table(), new Table(), new Human("Abhishek"))
assert(getNames(leggedThings) == List("Charles", "Sukanya", "Abhishek"), "Names don't match")

In [4]:
// Challenge option: reimplement with functors
def getNames(leggedThings : List[NumberOfLegs]): List[String] = {
    // begin solution
    leggedThings filter {
        leggedThing => leggedThing.isInstanceOf[Human]
    } map {
        leggedThing => leggedThing.asInstanceOf[Human]
    } map {
        human => human.getName
    } 
    // end solution
}

defined function getNames

In [5]:
// TEST CASE
val leggedThings : List[NumberOfLegs] = List(new Human("Charles"), new Human("Sukanya"), new Table(), new Table(), new Human("Abhishek"))
assert(getNames(leggedThings) == List("Charles", "Sukanya", "Abhishek"), "Names don't match")

leggedThings: List[NumberOfLegs] = List(
  ammonite.$sess.cmd1$Helper$Human@64995226,
  ammonite.$sess.cmd1$Helper$Human@674fb721,
  ammonite.$sess.cmd2$Helper$Table@4e29da44,
  ammonite.$sess.cmd2$Helper$Table@1564ff3b,
  ammonite.$sess.cmd1$Helper$Human@62f73263
)

In [7]:
// Challenge option: reimplement with functors
// ALTERNATIVE SOLUTION
def getNames(leggedThings : List[NumberOfLegs]): List[String] = {
    // begin solution
    (leggedThings foldRight[List[String]] (Nil)){
        case (h, acc) if h.isInstanceOf[Human] => 
                h.asInstanceOf[Human].getName :: acc
        case (_, acc) => acc
    } 
    // end solution
}
// TEST CASE
val leggedThings : List[NumberOfLegs] = List(new Human("Charles"), new Human("Sukanya"), new Table(), new Table(), new Human("Abhishek"))
assert(getNames(leggedThings) == List("Charles", "Sukanya", "Abhishek"), "Names don't match")

defined function getNames
leggedThings: List[NumberOfLegs] = List(
  ammonite.$sess.cmd1$Helper$Human@1a30e30d,
  ammonite.$sess.cmd1$Helper$Human@1d054b03,
  ammonite.$sess.cmd2$Helper$Table@760b482c,
  ammonite.$sess.cmd2$Helper$Table@6b3ee56b,
  ammonite.$sess.cmd1$Helper$Human@58464477
)

## Generics

Generic classes and functions are classes that take a type as a parameter.
We put these type parameters in square brackets (for example `List[String]` or `asInstanceOf[Human]` where we are passing `String` or `Human`, respectively)

One common use case for generics is for collections like maps, trees, and lists.
We will further exam generics using the example of lists.

If we wanted to define a list of integers, we could do something like this:

In [ ]:
sealed trait IntList
case class IntNil() extends IntList
case class IntCons(head: Int, tail: IntList) extends IntList

But what if we needed a list of strings instead?
We could write the following:

In [ ]:
sealed trait StringList
case class StringNil() extends StringList
case class StringCons(head: String, tail: StringList) extends StringList

What about booleans? Or your custom class you wrote, such as `Human` above?
We don't want to redefine these traits and the functions that work with them everytime we need a list for a different type.
We could define a list holding objects of type `Any` (the supertype of all types), but this has problems when we try access the objects we put in the list.

Instead we use generics.
We do this by adding type parameters to our code.

Coming back to our list example, we could write it using generics as follows:

In [ ]:
sealed trait GenericList[T]

case class GenericNil[T]() extends GenericList[T]
case class GenericCons[T](h: T, t: GenericList[T]) extends GenericList[T]

The type parameter `[T]` can be replaced with any class we want, either explicitly or by the type scala infers:

In [ ]:
val explicit = GenericCons[String]("hello", GenericCons[String]("world", GenericNil[String]()))
val inferred = GenericCons("hello", GenericCons("world", GenericNil()))

In [ ]:
val l = GenericCons[Integer](1, GenericNil())

### Exercise
As an exercise with our `GenericList`, let's define the map function for `GenericList`s

In [ ]:
def map[A,B](list: GenericList[A], function: (A) => B): GenericList[B] = {
    // begin solution
    list match {
        case GenericCons(h, t) => GenericCons[B](function(h), map(t, function))
        case GenericNil() => GenericNil[B]()
    }
    // end solution
}

In [ ]:
val newL = map[Integer, String](l, x => s"${x}")

In [ ]:
newL

### Type Bounds (If time permits)
We can also apply restrictions on the the types used for our type parameters

We can use `<:` to ensure that the type parameter is a subtype of the given type, or `:>` to ensure that the type parameter is a supertype of the given type.

In [ ]:
sealed trait LegList[T <: NumberOfLegs]

case class LegNil[T <: NumberOfLegs]() extends LegList[T]
case class LegCons[T <: NumberOfLegs](h: T, t: LegList[T]) extends LegList[T]

In [ ]:
def countLegs[T <: NumberOfLegs](leggedThings : LegList[T]) : Int = leggedThings match {
    case LegNil() => 0
    case LegCons(h, t) => h.getNumberOfLegs + countLegs(t)
}

In [ ]:
def countLegs[T <: NumberOfLegs](leggedThings : List[T]): Int = {
    leggedThings.foldLeft[Int](0)((accum, leggedThing) => accum + leggedThing.getNumberOfLegs)
}

Note that the ever useful `foldLeft` function also has a type parameter, which lets `foldLeft` return any type we want.  Of course, the compiler usually infers the type parameter's value, so we don't need include the type.

## That's all folks!